In [6]:
import numpy as np
import networkx as nx
import cvxpy as cp
import pickle

# Reading the Saved Random Graph
with open('random_graph.pkl', 'rb') as f:
    G, n, p = pickle.load(f)

# Retrieving Node Weights
w = {str(node): G.nodes[node]['weight'] for node in G.nodes()}

print(f'The Vertex Set of the Graph Is：{list(G.nodes)}')
print(f'The Edge Set of the Graph Is：{list(G.edges)}')
print(f'Node Weights: {w}')

# Greedy Algorithm
def greedy_mis(graph, w):
    i_set = set()
    sorted_graph = sorted(graph, key=lambda node: w[str(node)], reverse=True)
    for v in sorted_graph:
        i_set.add(v)
        if len(graph[v]) != 0:
            for w in graph[v]:
                if w in sorted_graph:
                    sorted_graph.remove(w)
    return i_set

# Converting Graph to Adjacency Matrix
nodes = list(G.nodes())
num_nodes = len(nodes)
adj = np.zeros((num_nodes, num_nodes), dtype=int)
for edge in G.edges():
    node1, node2 = edge
    index1 = nodes.index(node1)
    index2 = nodes.index(node2)
    adj[index1, index2] = 1
    adj[index2, index1] = 1

# Converting Adjacency Matrix to Adjacency List
graph = dict([])
for i in range(num_nodes):
    graph[i] = []
    for j in range(num_nodes):
        if adj[i, j] == 1:
            graph[i].append(j)

# Solving Maximum Independent Set Using Greedy Algorithm
greedy_is = greedy_mis(graph, w)
n = len(G.nodes)
binary_representation = [1 if i in greedy_is else 0 for i in range(n)]
binary_representation.reverse()
binary_string = ''.join(map(str, binary_representation))
A1 = binary_string
a1 = len(greedy_is)
Lower = 0
for i in range(len(greedy_is)):
    Lower += w[str(greedy_is.pop())]
print('Lower Bound Found by the Greedy Algorithm：', A1)
print('The Number of Lower Bound Points K1 Found by the Greedy Algorithm Is：', a1)
print('The Lower Bound Weight Found by the Greedy Algorithm Is：', Lower)

# Obtaining the Upper Bound a2 Using the Lovász Function
C = np.ones((len(G.nodes), len(G.nodes))) * max(w.values())
X = cp.Variable((n, n), PSD=True)
constraints = [cp.trace(X) == 1]
for i in range(len(adj)):
    for j in range(len(adj[0])):
        if adj[i][j] == 1:
            constraints.append(X[i][j] == 0)
prob = cp.Problem(cp.Maximize(cp.trace(C @ X)), constraints)
prob.solve()
print(X.value)
Upper = round(prob.value)
a2 = Upper // max(w.values())
print('The Number of Upper Bound Points Kl Found by Semidefinite Programming Is：', a2)
print('The Upper Bound Weight Found by Semidefinite Programming Is：', Upper)

The Vertex Set of the Graph Is：[0, 1, 2, 3, 4, 5, 6, 7]
The Edge Set of the Graph Is：[(0, 2), (0, 4), (0, 6), (0, 7), (1, 3), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 5), (3, 7), (4, 5), (4, 6), (4, 7), (6, 7)]
Node Weights: {'0': 2, '1': 2, '2': 6, '3': 10, '4': 13, '5': 5, '6': 3, '7': 14}
Lower Bound Found by the Greedy Algorithm： 10100000
The Number of Lower Bound Points K1 Found by the Greedy Algorithm Is： 2
The Lower Bound Weight Found by the Greedy Algorithm Is： 19
[[3.33330873e-01 3.33329421e-01 6.86266386e-12 1.52758164e-08
  7.04618888e-12 3.33329421e-01 7.04618914e-12 6.86266421e-12]
 [3.33329421e-01 3.33330887e-01 1.66997882e-08 6.12496470e-12
  1.70540440e-08 3.33329428e-01 7.04618930e-12 6.86266450e-12]
 [6.86266386e-12 1.66997882e-08 1.47008595e-06 3.89729906e-12
  3.89729910e-12 6.86266408e-12 3.89729918e-12 3.89729908e-12]
 [1.52758164e-08 6.12496470e-12 3.89729906e-12 1.47008595e-06
  1.09757816e-08 6.12496491e-12 1.09757816e-08 3.89729893e-12]
 [7.